In [3]:
import pandas as pd
import numpy as np
import random
import random
import string
import json

In [4]:
df_HIT_reviews = pd.read_csv('df_HIT_reviews.csv')
df_HIT_reviews

df_HITs = pd.read_csv('HITs.csv')
df_HITs

UNITS= df_HIT_reviews.shape[0] ## nr di righe nel df

## Un singolo Hit è generato  in base a una riga del data-frame contenente gli Hits (df_HITs)
## unit_id: identificativo della task: stringa
## token.input: token casuale: stringa
## token.output: token casuale: stringa
## documents: array contente singoli "documento" rappresentati una HIT
##
##
## struttura del singolo documento
## 
## documento = {
##             "id": asin,
##             "review_time": , 
##             "product_title": , 
##             "product_brand": ,
##             "review_text": 
##         }
## 
class Task:
    
    def __init__(self, i, my_df):
        self.index = i
        self.unit_id = self.set_unit_id(my_df, i)
        self.token_input = self.set_token()
        self.token_output = self.set_token()
        self.documents = self.set_documents(df=df_HITs)
    
    ## ricerca unit_id    
    ## df: df dove si trova unit_id
    ## row: riga dove si trova unit_id
    def set_unit_id(self, df, row):
        unit_id_value = df.at[row, 'HIT']
        return unit_id_value
    
    ## genera un token casuale composto da 11 caratteri maiuscoli
    def set_token(self):
        lunghezza = 11
        caratteri_maiuscoli = string.ascii_uppercase
        stringa_generata = ''.join(random.choice(caratteri_maiuscoli) for _ in range(lunghezza))
        return stringa_generata
    
    
    ## setta i documenti presenti all'interno dell'array documents
    def set_documents(self, df):
        
        documents=[]
        
        asins = df.iloc[self.index, 1:]
        
        for asin in asins:
            document = self.__document(df=df_HIT_reviews, asin=asin)
            documents.append(document)
        
        return documents
    
    ## Private
    ## estrae i dati dal df contenente le reviews
    def __document(self, df, asin):

        documento = {
            "id": asin,
            "review_time":df.loc[df['product_asin'] == asin, 'review_time'].values[0], 
            "product_title": df.loc[df['product_asin'] == asin, 'product_title'].values[0], 
            "product_brand": df.loc[df['product_asin'] == asin, 'product_brand'].values[0],
            "review_text": df.loc[df['product_asin'] == asin, 'review_text'].values[0]
        }
        return documento

In [5]:
def create_task(task):
        
    data = {
        "unit_id": task.unit_id,
        "token_input": task.token_input,
        "token_output": task.token_output,
        "documents_number": 5,
        "documents": task.documents
    }

    return data

    

In [6]:
json_finale = json.dumps({}, indent=2)

HITS = [] 

for i in range(0,UNITS):
    hit= Task(i, my_df=df_HITs)
    data = create_task(hit)
    
    HITS.append(data)
    
with open('Crowd_Frame riscaricato/data/build/task/hits.json', 'w') as json_file:
    json.dump(HITS, json_file)